In [ ]:
%install '.package(path: "$cwd/FastaiNotebook_06_cuda")' FastaiNotebook_06_cuda

Installing packages:
	.package(path: "/usr/local/google/home/jekbradbury/fastai_docs/dev_swift/FastaiNotebook_06_cuda")
		FastaiNotebook_06_cuda
With SwiftPM flags: []
Working in: /tmp/tmp174nwyrf
Fetching https://github.com/JustHTTP/Just
Fetching https://github.com/mxcl/Path.swift
Completed resolution in 4.17s
Cloning https://github.com/JustHTTP/Just
Resolving https://github.com/JustHTTP/Just at 0.7.1
Cloning https://github.com/mxcl/Path.swift
Resolving https://github.com/mxcl/Path.swift at 0.16.2
Compile Swift Module 'Just' (1 sources)
Compile Swift Module 'Path' (9 sources)
Compile Swift Module 'FastaiNotebook_06_cuda' (10 sources)
Compile Swift Module 'jupyterInstalledPackages' (1 sources)
Linking ./.build/x86_64-unknown-linux/debug/libjupyterInstalledPackages.so
Initializing Swift...
Loading library...
Installation complete!


## Load data

In [ ]:
import FastaiNotebook_06_cuda
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

('inline', 'module://ipykernel.pylab.backend_inline')


In [ ]:
// export
import Path
import TensorFlow
import Python

In [ ]:
let plt = Python.import("matplotlib.pyplot")

In [ ]:
let data = mnistDataBunch(flat: false, bs: 512)

2019-04-12 17:29:26.652513: W tensorflow/core/framework/allocator.cc:122] Allocation of 188160000 exceeds 10% of system memory.
2019-04-12 17:29:26.743076: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2019-04-12 17:29:26.772550: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3700000000 Hz
2019-04-12 17:29:26.779184: W tensorflow/core/framework/allocator.cc:122] Allocation of 188160000 exceeds 10% of system memory.


In [ ]:
let opt = SGD<CnnModel, Float>(learningRate: 0.4)
func modelInit() -> CnnModel { return CnnModel(sizeIn:28, channelIn: 1, channelOut: 10, nFilters: [8, 16, 32, 32]) }
let learner = Learner(data: data, lossFunction: softmaxCrossEntropy, optimizer: opt, initializingWith: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])

In [ ]:
time { try! learner.fit(1) }

Epoch 0: [0.36644837, 0.8903]                                                   
14150.869651 ms                                                               


## Batchnorm

### Custom

Let's start by building our own `BatchNorm` layer from scratch. Eventually we intend for this code to do the trick:

In [ ]:
struct AlmostBatchNorm<Scalar: TensorFlowFloatingPoint>: Differentiable {
    // Configuration hyperparameters
    @noDerivative let momentum: Scalar
    @noDerivative let epsilon: Scalar
    // Running statistics
    @noDerivative var runningMean: Tensor<Scalar>
    @noDerivative var runningVariance: Tensor<Scalar>
    // Trainable parameters
    var scale: Tensor<Scalar>
    var offset: Tensor<Scalar>
    
    init(featureCount: Int, momentum: Scalar = 0.9, epsilon: Scalar = 1e-5) {
        self.momentum = momentum
        self.epsilon = epsilon
        self.scale = Tensor(ones: [Int32(featureCount)])
        self.offset = Tensor(zeros: [Int32(featureCount)])
        self.runningMean = Tensor(0)
        self.runningVariance = Tensor(1)
    }

    mutating func applied(to input: Tensor<Scalar>, in context: Context) -> Tensor<Scalar> {
        let mean: Tensor<Scalar>
        let variance: Tensor<Scalar>
        switch context.learningPhase {
        case .training:
            mean = input.mean(alongAxes: [0, 1, 2])
            variance = input.variance(alongAxes: [0, 1, 2])
            runningMean += (mean - runningMean) * (1 - momentum)
            runningVariance += (variance - runningVariance) * (1 - momentum)
        case .inference:
            mean = runningMean
            variance = runningVariance
        }
        let normalizer = rsqrt(variance + epsilon) * scale
        return (input - mean) * normalizer + offset
    }
}

But there are some automatic differentiation limitations (control flow support) and `Layer` protocol constraints (mutating `applied`) that make this impossible for now (note the lack of `@differentiable` or a `Layer` conformance), so we'll need a few workarounds. A `Reference` will let us update running statistics without declaring the `applied` method `mutating`:

In [ ]:
//export
class Reference<T> {
    var value: T
    init(_ value: T) {
        self.value = value
    }
}

The following snippet will let us differentiate a layer's `applied` method if it's composed of training and inference implementations that are each differentiable:

In [ ]:
//export
protocol LearningPhaseDependent: Layer {
    @differentiable func applyingTraining(to input: Input) -> Output
    @differentiable func applyingInference(to input: Input) -> Output
}

extension LearningPhaseDependent {
    func applied(to input: Input, in context: Context) -> Output {
        switch context.learningPhase {
        case .training: return applyingTraining(to: input)
        case .inference: return applyingInference(to: input)
        }
    }

    @differentiating(applied)
    func gradApplied(to input: Input, in context: Context) ->
        (value: Output, pullback: (Output.CotangentVector) ->
            (Self.CotangentVector, Input.CotangentVector)) {
        switch context.learningPhase {
        case .training:
            return valueWithPullback(at: input) {
                $0.applyingTraining(to: $1)
            }
        case .inference:
            return valueWithPullback(at: input) {
                $0.applyingInference(to: $1)
            }
        }
    }
}

Now we can implement a BatchNorm that we can use in our models:

In [ ]:
//export
protocol Norm: Layer {
    associatedtype Scalar
    typealias Input = Tensor<Scalar>
    typealias Output = Tensor<Scalar>
    init(featureCount: Int, epsilon: Scalar)
}

struct FABatchNorm<Scalar: TensorFlowFloatingPoint>: LearningPhaseDependent, Norm {
    // Configuration hyperparameters
    @noDerivative let momentum: Scalar
    @noDerivative let epsilon: Scalar
    // Running statistics
    @noDerivative let runningMean: Reference<Tensor<Scalar>>
    @noDerivative let runningVariance: Reference<Tensor<Scalar>>
    // Trainable parameters
    var scale: Tensor<Scalar>
    var offset: Tensor<Scalar>
    // TODO: check why these aren't being synthesized
    typealias Input = Tensor<Scalar>
    typealias Output = Tensor<Scalar>
    
    init(featureCount: Int, momentum: Scalar, epsilon: Scalar = 1e-5) {
        self.momentum = momentum
        self.epsilon = epsilon
        self.scale = Tensor(ones: [Int32(featureCount)])
        self.offset = Tensor(zeros: [Int32(featureCount)])
        self.runningMean = Reference(Tensor(0))
        self.runningVariance = Reference(Tensor(1))
    }
    
    init(featureCount: Int, epsilon: Scalar = 1e-5) {
        self.init(featureCount: featureCount, momentum: 0.9, epsilon: epsilon)
    }

    @differentiable
    func applyingTraining(to input: Tensor<Scalar>) -> Tensor<Scalar> {
        let mean = input.mean(alongAxes: [0, 1, 2])
        let variance = input.variance(alongAxes: [0, 1, 2])
        runningMean.value += (mean - runningMean.value) * (1 - momentum)
        runningVariance.value += (variance - runningVariance.value) * (1 - momentum)
        let normalizer = rsqrt(variance + epsilon) * scale
        return (input - mean) * normalizer + offset
    }
    
    @differentiable
    func applyingInference(to input: Tensor<Scalar>) -> Tensor<Scalar> {
        let mean = runningMean.value
        let variance = runningVariance.value
        let normalizer = rsqrt(variance + epsilon) * scale
        return (input - mean) * normalizer + offset
    }
}

//export
struct ConvBN<Scalar: TensorFlowFloatingPoint>: Layer {
    var conv: Conv2D<Scalar>
    var norm: FABatchNorm<Scalar>
    init(
        filterShape: (Int, Int, Int, Int),
        strides: (Int, Int) = (1, 1),
        padding: Padding = .valid,
        activation: @escaping Conv2D<Scalar>.Activation = identity
    ) {
        // TODO (when control flow AD works): use Conv2D without bias
        self.conv = Conv2D(
            filterShape: filterShape,
            strides: strides,
            padding: padding,
            activation: activation)
        self.norm = FABatchNorm(featureCount: filterShape.3, epsilon: 1e-5)
    }

    @differentiable
    func applied(to input: Tensor<Scalar>, in context: Context) -> Tensor<Scalar> {
        return norm.applied(to: conv.applied(to: input, in: context), in: context)
    }
}

In [ ]:
// Would be great if this generic could work
// struct ConvNorm<NormType: Norm, Scalar: TensorFlowFloatingPoint>: Layer
//     where NormType.Scalar == Scalar {
//     var conv: Conv2D<Scalar>
//     var norm: NormType
//     init(
//         filterShape: (Int, Int, Int, Int),
//         strides: (Int, Int) = (1, 1),
//         padding: Padding = .valid,
//         activation: @escaping Conv2D<Scalar>.Activation = identity
//     ) {
//         // TODO (when control flow AD works): use Conv2D without bias
//         self.conv = Conv2D(
//             filterShape: filterShape,
//             strides: strides,
//             padding: padding,
//             activation: activation)
//         self.norm = NormType.init(featureCount: filterShape.3, epsilon: 1e-5)
//     }

//     @differentiable
//     func applied(to input: Tensor<Scalar>, in context: Context) -> Tensor<Scalar> {
//         return norm.applied(to: conv.applied(to: input, in: context), in: context)
//     }
// }
//typealias ConvBN = ConvNorm<BatchNorm<Float>, Float>

In [ ]:
//export 
public struct CnnModelBN: Layer {
    public var reshapeToSquare: Reshape<Float>
    public var conv1: ConvBN<Float>
    public var conv2: ConvBN<Float>
    public var conv3: ConvBN<Float>
    public var conv4: ConvBN<Float>
    public var pool = AvgPool2D<Float>(poolSize: (2, 2), strides: (1, 1))
    public var flatten = Flatten<Float>()
    public var linear: Dense<Float>
    
    public init(
        sizeIn: Int,
        channelIn: Int,
        channelOut: Int,
        nFilters: [Int]
    ) {
        reshapeToSquare = Reshape<Float>(
            [-1, Int32(sizeIn), Int32(sizeIn), Int32(channelIn)])
        conv1 = ConvBN<Float>(
            filterShape: (5, 5, 1, nFilters[0]), 
            strides: (2, 2), 
            padding: .same, 
            activation: relu)
        conv2 = ConvBN<Float>(
            filterShape: (3, 3, nFilters[0], nFilters[1]),
            strides: (2, 2),
            padding: .same,
            activation: relu)
        conv3 = ConvBN<Float>(
            filterShape: (3, 3, nFilters[1], nFilters[2]),
            strides: (2, 2),
            padding: .same,
            activation: relu)
        conv4 = ConvBN<Float>(
            filterShape: (3, 3, nFilters[2], nFilters[3]),
            strides: (2, 2),
            padding: .same,
            activation: relu)
        linear = Dense<Float>(inputSize: nFilters[3], outputSize: channelOut)
    }
    
    @differentiable
    public func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        // There isn't a "sequenced" defined with enough layers.
        let intermediate =  input.sequenced(
            in: context,
            through: reshapeToSquare, conv1, conv2, conv3, conv4)
        return intermediate.sequenced(in: context, through: pool, flatten, linear)
    }
}

In [ ]:
let opt = SGD<CnnModelBN, Float>(learningRate: 0.4)
func modelInit() -> CnnModelBN { return CnnModelBN(sizeIn:28, channelIn: 1, channelOut: 10, nFilters: [8, 16, 32, 32]) }
let learner = Learner(data: data, lossFunction: softmaxCrossEntropy, optimizer: opt, initializingWith: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])

In [ ]:
time { try! learner.fit(1) }

Epoch 0: [0.12745029, 0.963]                                                    
17907.907068 ms                                                               


TODO: hooks/LayerDelegates

## More norms

### Layer norm

From [the paper](https://arxiv.org/abs/1607.06450): "*batch normalization cannot be applied to online learning tasks or to extremely large distributed models where the minibatches have to be small*".

General equation for a norm layer with learnable affine:

$$y = \frac{x - \mathrm{E}[x]}{ \sqrt{\mathrm{Var}[x] + \epsilon}} * \gamma + \beta$$

The difference with BatchNorm is
1. we don't keep a moving average
2. we don't average over the batches dimension but over the hidden dimension, so it's independent of the batch size

In [ ]:
struct LayerNorm2D<Scalar: TensorFlowFloatingPoint>: Norm {
    // Configuration hyperparameters
    @noDerivative let epsilon: Scalar
    // Trainable parameters
    var scale: Tensor<Scalar>
    var offset: Tensor<Scalar>
    
    init(featureCount: Int, epsilon: Scalar = 1e-5) {
        self.epsilon = epsilon
        self.scale = Tensor(ones: [Int32(featureCount)])
        self.offset = Tensor(zeros: [Int32(featureCount)])
    }
    
    @differentiable
    func applied(to input: Tensor<Scalar>, in context: Context) -> Tensor<Scalar> {
        let mean = input.mean(alongAxes: [1, 2, 3])
        let variance = input.variance(alongAxes: [1, 2, 3])
        let normalizer = rsqrt(variance + epsilon) * scale
        return (input - mean) * normalizer + offset
    }
}

struct ConvLN<Scalar: TensorFlowFloatingPoint>: Layer {
    var conv: Conv2D<Scalar>
    var norm: LayerNorm2D<Scalar>
    init(
        filterShape: (Int, Int, Int, Int),
        strides: (Int, Int) = (1, 1),
        padding: Padding = .valid,
        activation: @escaping Conv2D<Scalar>.Activation = identity
    ) {
        // TODO (when control flow AD works): use Conv2D without bias
        self.conv = Conv2D(
            filterShape: filterShape,
            strides: strides,
            padding: padding,
            activation: activation)
        self.norm = LayerNorm2D(featureCount: filterShape.3, epsilon: 1e-5)
    }

    @differentiable
    func applied(to input: Tensor<Scalar>, in context: Context) -> Tensor<Scalar> {
        return norm.applied(to: conv.applied(to: input, in: context), in: context)
    }
}

In [ ]:
struct CnnModelLN: Layer {
    var reshapeToSquare = Reshape<Float>([-1, 28, 28, 1])
    var conv1 = ConvLN<Float>(
        filterShape: (5, 5, 1, 8),
        strides: (2, 2),
        padding: .same,
        activation: relu)
    var conv2 = ConvLN<Float>(
        filterShape: (3, 3, 8, 16),
        strides: (2, 2),
        padding: .same,
        activation: relu)
    var conv3 = ConvLN<Float>(
        filterShape: (3, 3, 16, 32),
        strides: (2, 2),
        padding: .same,
        activation: relu)
    var conv4 = ConvLN<Float>(
        filterShape: (3, 3, 32, 32),
        strides: (2, 2),
        padding: .same,
        activation: relu)
    
    var pool = AvgPool2D<Float>(poolSize: (2, 2), strides: (1, 1))
    
    var flatten = Flatten<Float>()
    var linear = Dense<Float>(inputSize: 32, outputSize: 10)
    
    @differentiable
    func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        // There isn't a "sequenced" defined with enough layers.
        let intermediate =  input.sequenced(
            in: context,
            through: reshapeToSquare, conv1, conv2, conv3, conv4)
        return intermediate.sequenced(in: context, through: pool, flatten, linear)
    }
}

let opt = SGD<CnnModelLN, Float>(learningRate: 0.4)
func modelInit() -> CnnModelLN { return CnnModelLN() }
let learner = Learner(data: data, lossFunction: softmaxCrossEntropy, optimizer: opt, initializingWith: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])

time { try! learner.fit(1) }

Epoch 0: [0.18734914, 0.9456]                                                   
17647.93642 ms                                                                


In [ ]:
struct InstanceNorm<Scalar: TensorFlowFloatingPoint>: Norm {
    // Configuration hyperparameters
    @noDerivative let epsilon: Scalar
    // Trainable parameters
    var scale: Tensor<Scalar>
    var offset: Tensor<Scalar>
    
    init(featureCount: Int, epsilon: Scalar = 1e-5) {
        self.epsilon = epsilon
        self.scale = Tensor(ones: [Int32(featureCount)])
        self.offset = Tensor(zeros: [Int32(featureCount)])
    }
    
    @differentiable
    func applied(to input: Tensor<Scalar>, in context: Context) -> Tensor<Scalar> {
        let mean = input.mean(alongAxes: [2, 3])
        let variance = input.variance(alongAxes: [2, 3])
        let normalizer = rsqrt(variance + epsilon) * scale
        return (input - mean) * normalizer + offset
    }
}

struct ConvIN<Scalar: TensorFlowFloatingPoint>: Layer {
    var conv: Conv2D<Scalar>
    var norm: InstanceNorm<Scalar>
    init(
        filterShape: (Int, Int, Int, Int),
        strides: (Int, Int) = (1, 1),
        padding: Padding = .valid,
        activation: @escaping Conv2D<Scalar>.Activation = identity
    ) {
        // TODO (when control flow AD works): use Conv2D without bias
        self.conv = Conv2D(
            filterShape: filterShape,
            strides: strides,
            padding: padding,
            activation: activation)
        self.norm = InstanceNorm(featureCount: filterShape.3, epsilon: 1e-5)
    }

    @differentiable
    func applied(to input: Tensor<Scalar>, in context: Context) -> Tensor<Scalar> {
        return norm.applied(to: conv.applied(to: input, in: context), in: context)
    }
}

Lost in all those norms? The authors from the [group norm paper](https://arxiv.org/pdf/1803.08494.pdf) have you covered:

![Various norms](../dev_course/dl2/images/norms.png)

TODO/skipping GroupNorm

### Running Batch Norm

In [ ]:
struct RunningBatchNorm<Scalar: TensorFlowFloatingPoint>: LearningPhaseDependent, Norm {
    // Configuration hyperparameters
    @noDerivative let momentum: Scalar
    @noDerivative let epsilon: Scalar
    // Running statistics
    @noDerivative let runningSum: Reference<Tensor<Scalar>>
    @noDerivative let runningSumOfSquares: Reference<Tensor<Scalar>>
    @noDerivative let runningCount: Reference<Scalar>
    @noDerivative let samplesSeen: Reference<Int32>
    // Trainable parameters
    var scale: Tensor<Scalar>
    var offset: Tensor<Scalar>
    // TODO: check why these aren't being synthesized
    typealias Input = Tensor<Scalar>
    typealias Output = Tensor<Scalar>
    
    init(featureCount: Int, momentum: Scalar, epsilon: Scalar = 1e-5) {
        self.momentum = momentum
        self.epsilon = epsilon
        self.scale = Tensor(ones: [Int32(featureCount)])
        self.offset = Tensor(zeros: [Int32(featureCount)])
        self.runningSum = Reference(Tensor(0))
        self.runningSumOfSquares = Reference(Tensor(0))
        self.runningCount = Reference(Scalar(0))
        self.samplesSeen = Reference(0)
    }
    
    init(featureCount: Int, epsilon: Scalar = 1e-5) {
        self.init(featureCount: featureCount, momentum: 0.9, epsilon: epsilon)
    }

    @differentiable
    func applyingTraining(to input: Tensor<Scalar>) -> Tensor<Scalar> {
        let (batch, channels) = (input.shape[0], Scalar(input.shape[3]))
        let sum = input.sum(alongAxes: [0, 1, 2])
        let sumOfSquares = (input * input).sum(alongAxes: [0, 1, 2])
        let count = Scalar(input.scalarCount).withoutDerivative() / channels
        let mom = momentum / sqrt(Scalar(batch) - 1)
        let runningSum = mom * self.runningSum.value + (1 - mom) * sum
        let runningSumOfSquares = mom * self.runningSumOfSquares.value + (
            1 - mom) * sumOfSquares
        let runningCount = mom * self.runningCount.value + (1 - mom) * count
        
        self.runningSum.value = runningSum
        self.runningSumOfSquares.value = runningSumOfSquares
        self.runningCount.value = runningCount
        self.samplesSeen.value += batch
        
        let mean = runningSum / runningCount
        let variance = runningSumOfSquares / runningCount - mean * mean
        
        let normalizer = rsqrt(variance + epsilon) * scale
        return (input - mean) * normalizer + offset
    }
    
    @differentiable
    func applyingInference(to input: Tensor<Scalar>) -> Tensor<Scalar> {
        let mean = runningSum.value / runningCount.value
        let variance = runningSumOfSquares.value / runningCount.value - mean * mean
        let normalizer = rsqrt(variance + epsilon) * scale
        return (input - mean) * normalizer + offset
    }
}

struct ConvRBN<Scalar: TensorFlowFloatingPoint>: Layer {
    var conv: Conv2D<Scalar>
    var norm: RunningBatchNorm<Scalar>
    init(
        filterShape: (Int, Int, Int, Int),
        strides: (Int, Int) = (1, 1),
        padding: Padding = .valid,
        activation: @escaping Conv2D<Scalar>.Activation = identity
    ) {
        // TODO (when control flow AD works): use Conv2D without bias
        self.conv = Conv2D(
            filterShape: filterShape,
            strides: strides,
            padding: padding,
            activation: activation)
        self.norm = RunningBatchNorm(featureCount: filterShape.3, epsilon: 1e-5)
    }

    @differentiable
    func applied(to input: Tensor<Scalar>, in context: Context) -> Tensor<Scalar> {
        return norm.applied(to: conv.applied(to: input, in: context), in: context)
    }
}

In [ ]:
struct CnnModelRBN: Layer {
    var reshapeToSquare = Reshape<Float>([-1, 28, 28, 1])
    var conv1 = ConvRBN<Float>(
        filterShape: (5, 5, 1, 8),
        strides: (2, 2),
        padding: .same,
        activation: relu)
    var conv2 = ConvRBN<Float>(
        filterShape: (3, 3, 8, 16),
        strides: (2, 2),
        padding: .same,
        activation: relu)
    var conv3 = ConvRBN<Float>(
        filterShape: (3, 3, 16, 32),
        strides: (2, 2),
        padding: .same,
        activation: relu)
    var conv4 = ConvRBN<Float>(
        filterShape: (3, 3, 32, 32),
        strides: (2, 2),
        padding: .same,
        activation: relu)
    
    var pool = AvgPool2D<Float>(poolSize: (2, 2), strides: (1, 1))
    
    var flatten = Flatten<Float>()
    var linear = Dense<Float>(inputSize: 32, outputSize: 10)
    
    @differentiable
    func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        // There isn't a "sequenced" defined with enough layers.
        let intermediate =  input.sequenced(
            in: context,
            through: reshapeToSquare, conv1, conv2, conv3, conv4)
        return intermediate.sequenced(in: context, through: pool, flatten, linear)
    }
}

let opt = SGD<CnnModelRBN, Float>(learningRate: 0.4)
func modelInit() -> CnnModelRBN { return CnnModelRBN() }
let learner = Learner(data: data, lossFunction: softmaxCrossEntropy, optimizer: opt, initializingWith: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])

time { try! learner.fit(1) }

Epoch 0: [0.108368374, 0.9699]                                                  
18568.891171 ms                                                               


Glue to get XLA compilation and AD to work together:

In [ ]:
struct PullbackArgs<T : TensorGroup, U : TensorGroup> : TensorGroup {
  let input: T
  let cotangent: U
}

 func xlaCompiled<T : Differentiable & TensorGroup, U : Differentiable & TensorGroup>(
  _ fn: @escaping @differentiable (T) -> U) -> @differentiable (T) -> U
where T.CotangentVector : TensorGroup, U.CotangentVector : TensorGroup
{
  let xlaCompiledFn: (T) -> U = _graph(fn, useXla: true) // renaming to useXLA
  let xlaCompiledPullback = _graph(
    { (pbArgs: PullbackArgs<T, U.CotangentVector>) in
      pullback(at: pbArgs.input, in: fn)(pbArgs.cotangent)
    },
    useXla: true)
  return  differentiableFunction { x in
      (value: xlaCompiledFn(x),
      pullback: {
        v in
        xlaCompiledPullback(PullbackArgs(input: x, cotangent: v))}
      )
  }
}

In [ ]:
struct XLARunningBatchNorm<Scalar: TensorFlowFloatingPoint>: LearningPhaseDependent, Norm {
    // Configuration hyperparameters
    @noDerivative let momentum: Scalar
    @noDerivative let epsilon: Scalar
    // Running statistics
    @noDerivative let runningSum: Reference<Tensor<Scalar>>
    @noDerivative let runningSumOfSquares: Reference<Tensor<Scalar>>
    @noDerivative let runningCount: Reference<Scalar>
    @noDerivative let samplesSeen: Reference<Int32>
    // Trainable parameters
    var scale: Tensor<Scalar>
    var offset: Tensor<Scalar>
    // TODO: check why these aren't being synthesized
    typealias Input = Tensor<Scalar>
    typealias Output = Tensor<Scalar>
    
    // needed for tracing
    struct TrainingKernelInput: TensorGroup, Differentiable, AdditiveArithmetic { // needs AA?
        let input: Tensor<Scalar>
        let mom: Tensor<Scalar>
        let runningSum: Tensor<Scalar>
        let runningSumOfSquares: Tensor<Scalar>
        let newRunningCount: Tensor<Scalar>
        let scale: Tensor<Scalar>
        let offset: Tensor<Scalar>
        let epsilon: Tensor<Scalar>
    }
    
    struct TrainingKernelOutput: TensorGroup, Differentiable, AdditiveArithmetic {
        let normalized: Tensor<Scalar>
        let newRunningSum: Tensor<Scalar>
        let newRunningSumOfSquares: Tensor<Scalar>
    }
    
    static func trainingKernel(_ input: TrainingKernelInput) -> TrainingKernelOutput {
        let sum = input.input.sum(alongAxes: [0, 1, 2])
        let sumOfSquares = (input.input * input.input).sum(alongAxes: [0, 1, 2])
        let newRunningSum = input.mom * input.runningSum + (1 - input.mom) * sum
        let newRunningSumOfSquares = input.mom * input.runningSumOfSquares + (
            1 - input.mom) * sumOfSquares
        let mean = newRunningSum / input.newRunningCount
        let variance = newRunningSumOfSquares / input.newRunningCount - mean * mean
        let normalizer = rsqrt(variance + input.epsilon) * input.scale
        let normalized = (input.input - mean) * normalizer + input.offset
        return TrainingKernelOutput(
            normalized: normalized,
            newRunningSum: newRunningSum,
            newRunningSumOfSquares: newRunningSumOfSquares
        )
    }
    
    let compiledTrainingKernel: @differentiable (TrainingKernelInput) -> TrainingKernelOutput
    
    init(featureCount: Int, momentum: Scalar, epsilon: Scalar = 1e-5) {
        self.momentum = momentum
        self.epsilon = epsilon
        self.scale = Tensor(ones: [Int32(featureCount)])
        self.offset = Tensor(zeros: [Int32(featureCount)])
        self.runningSum = Reference(Tensor(0))
        self.runningSumOfSquares = Reference(Tensor(0))
        self.runningCount = Reference(Scalar(0))
        self.samplesSeen = Reference(0)
        // Compile the training kernel to a TensorFlow graph. TensorFlow will then
        // compile it to XLA once for each set of input shapes (hopefully!).
        self.compiledTrainingKernel = xlaCompiled(XLARunningBatchNorm<Scalar>.trainingKernel)
    }
    
    init(featureCount: Int, epsilon: Scalar = 1e-5) {
        self.init(featureCount: featureCount, momentum: 0.9, epsilon: epsilon)
    }
    
    @differentiable
    func applyingTraining(to input: Tensor<Scalar>) -> Tensor<Scalar> {
        let (batch, channels) = (input.shape[0], Scalar(input.shape[3]))
        // it's fine to do scalar computation outside the JIT
        let mom = momentum / sqrt(Scalar(batch) - 1)
        let count = Scalar(input.scalarCount).withoutDerivative() / channels
        let newRunningCount = mom * runningCount.value + (1 - mom) * count
        
        let kernelInput = TrainingKernelInput(
            input: input,
            mom: Tensor(mom),
            runningSum: runningSum.value,
            runningSumOfSquares: runningSumOfSquares.value,
            newRunningCount: Tensor(newRunningCount),
            scale: scale,
            offset: offset,
            epsilon: Tensor(epsilon)
        )

        let kernelOutput = compiledTrainingKernel(kernelInput)
        
        self.runningSum.value = kernelOutput.newRunningSum
        self.runningSumOfSquares.value = kernelOutput.newRunningSumOfSquares
        self.runningCount.value = newRunningCount
        self.samplesSeen.value += batch
        
        return kernelOutput.normalized
    }
    
    @differentiable
    func applyingInference(to input: Tensor<Scalar>) -> Tensor<Scalar> {
        let mean = runningSum.value / runningCount.value
        let variance = runningSumOfSquares.value / runningCount.value - mean * mean
        let normalizer = rsqrt(variance + epsilon) * scale
        return (input - mean) * normalizer + offset
    }
}

In [ ]:
struct ConvXRBN<Scalar: TensorFlowFloatingPoint>: Layer {
    var conv: Conv2D<Scalar>
    var norm: XLARunningBatchNorm<Scalar>
    init(
        filterShape: (Int, Int, Int, Int),
        strides: (Int, Int) = (1, 1),
        padding: Padding = .valid,
        activation: @escaping Conv2D<Scalar>.Activation = identity
    ) {
        // TODO (when control flow AD works): use Conv2D without bias
        self.conv = Conv2D(
            filterShape: filterShape,
            strides: strides,
            padding: padding,
            activation: activation)
        self.norm = XLARunningBatchNorm(featureCount: filterShape.3, epsilon: 1e-5)
    }

    @differentiable
    func applied(to input: Tensor<Scalar>, in context: Context) -> Tensor<Scalar> {
        return norm.applied(to: conv.applied(to: input, in: context), in: context)
    }
}

In [ ]:
struct CnnModelXRBN: Layer {
    var reshapeToSquare = Reshape<Float>([-1, 28, 28, 1])
    var conv1 = ConvXRBN<Float>(
        filterShape: (5, 5, 1, 8),
        strides: (2, 2),
        padding: .same,
        activation: relu)
    var conv2 = ConvXRBN<Float>(
        filterShape: (3, 3, 8, 16),
        strides: (2, 2),
        padding: .same,
        activation: relu)
    var conv3 = ConvXRBN<Float>(
        filterShape: (3, 3, 16, 32),
        strides: (2, 2),
        padding: .same,
        activation: relu)
    var conv4 = ConvXRBN<Float>(
        filterShape: (3, 3, 32, 32),
        strides: (2, 2),
        padding: .same,
        activation: relu)
    
    var pool = AvgPool2D<Float>(poolSize: (2, 2), strides: (1, 1))
    
    var flatten = Flatten<Float>()
    var linear = Dense<Float>(inputSize: 32, outputSize: 10)
    
    @differentiable
    func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        // There isn't a "sequenced" defined with enough layers.
        let intermediate =  input.sequenced(
            in: context,
            through: reshapeToSquare, conv1, conv2, conv3, conv4)
        return intermediate.sequenced(in: context, through: pool, flatten, linear)
    }
}

let opt = SGD<CnnModelXRBN, Float>(learningRate: 0.4)
func modelInit() -> CnnModelXRBN { return CnnModelXRBN() }
let learner = Learner(data: data, lossFunction: softmaxCrossEntropy, optimizer: opt, initializingWith: modelInit)
let recorder = learner.makeDefaultDelegates(metrics: [accuracy])

time { try! learner.fit(1) }

Fatal error: TF runtime assertion failure: file /swift-base/swift/stdlib/public/TensorFlow/TensorHandle.swift, line 128
Current stack trace:
0    libswiftCore.so                    0x00007fb67a285650 _swift_stdlib_reportFatalErrorInFile + 115
1    libswiftCore.so                    0x00007fb67a1ce2dc <unavailable> + 3089116
2    libswiftCore.so                    0x00007fb67a1ce3ce <unavailable> + 3089358
3    libswiftCore.so                    0x00007fb67a015342 <unavailable> + 1282882
4    libswiftCore.so                    0x00007fb67a19acf2 <unavailable> + 2878706
5    libswiftCore.so                    0x00007fb67a014789 <unavailable> + 1279881
6    libswiftTensorFlow.so              0x00007fb67744bab2 <unavailable> + 567986
7    libswiftTensorFlow.so              0x00007fb677472000 TensorHandle.makeHostCopy() + 131
8    libswiftTensorFlow.so              0x00007fb677471e9c <unavailable> + 724636
9    libswiftTensorFlow.so              0x00007fb677472990 __tf_get_scalar_or_die_Int

Blocked on https://github.com/apple/swift/pull/24009

## Export

In [ ]:
notebookToScript(fname: (Path.cwd / "07_batchnorm.ipynb").string)